In [4]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from torch_geometric.data import Data

path1 = Path('../../../czl/TwiBot22-baselines/datasets/Twibot-20')
path2 = Path('../../../czl/TwiBot22-baselines/data')
path3 = Path('../../../czl/TwiBot22-baselines/src')

edge = pd.read_csv(path1 / 'edge.csv')

# node_des = np.load('data/node_des.npy', allow_pickle=True).tolist()
# labeled_user_count = len(node_des)

In [2]:
nodes = pd.read_json(path1 / 'node.json')
users = nodes[nodes.id.str.contains('^u')]

user_index_to_uid = list(users.id)
uid_to_user_index = dict(map(reversed, enumerate(user_index_to_uid)))

In [3]:
edge = edge[edge.relation != 'post']
edge = edge.reset_index(drop=True)
edge.source_id = list(map(lambda x: uid_to_user_index[x], edge.source_id))
edge.target_id = list(map(lambda x: uid_to_user_index[x], edge.target_id))


In [17]:
edge_index = []
source_id = torch.tensor(edge.source_id)
target_id = torch.tensor(edge.target_id)
a = torch.cat((source_id, target_id), dim=-1)
b = torch.cat((target_id, source_id), dim=-1)
edge_index = torch.stack((a,b))

tensor(1)


In [19]:
torch.save(edge_index, 'data/edge_index.pt')

tensor(1) tensor(15234)


In [1]:

user_edge = {i: [] for i in range(len(users))}
for i in tqdm(range(len(edge.source_id))):
    user_edge[edge.source_id[i]].append([edge.source_id[i], edge.target_id[i]])
    user_edge[edge.source_id[i]].append([edge.target_id[i], edge.source_id[i]])
    user_edge[edge.target_id[i]].append([edge.source_id[i], edge.target_id[i]])
    user_edge[edge.target_id[i]].append([edge.target_id[i], edge.source_id[i]])

NameError: name 'users' is not defined

In [35]:
user_edge_new = {i: [] for i in range(11826)}

for i in tqdm(range(len(user_edge_new))):
    tmp = []
    for j in range(len(user_edge[i])):
        if user_edge[i][j] not in tmp:
            tmp.append(user_edge[i][j])
    tmp = np.array(tmp)
    user_edge_new[i] = tmp.transpose().tolist()

100%|██████████| 11826/11826 [00:02<00:00, 5267.97it/s]


[11825, 11803, 229355, 11825, 228571, 11825, 229014, 11825, 227889, 11825, 225381, 11825, 224136, 11825, 229348, 11825, 224556, 11825, 224380, 11825, 8166, 11825, 89383, 11825, 229399, 11825, 226402, 11825, 229110, 11825, 11802, 11825, 224618, 11825, 228554, 11825, 225185, 11825, 229050, 11825, 224744, 11825]


In [18]:
user_edge_final = {i:[] for i in range(len(user_edge_new))}
for i in tqdm(range(len(user_edge_new))):
    if user_edge_new[i] != []:
        user_order = list(set(user_edge_new[i][0]))
        for j in range(len(user_order)):
            if user_order[j] == i:
                user_order[0], user_order[j] = user_order[j], user_order[0]
        user_edge_final[i] = user_order
    else:
        user_order = [i]
        user_edge_final[i] = user_order

100%|██████████| 11826/11826 [00:00<00:00, 112457.98it/s]


In [20]:
print(user_edge_final[0])

[0, 3393, 4389, 649, 1131, 2507, 5196, 3118, 9452, 10994, 9205, 9526, 151, 4120, 4217, 97723, 2300, 5789]


In [21]:
np.save('data/user_neighbor_index.npy', user_edge_final)

In [90]:
users_num_pro = torch.load(path2 / 'num_properties_tensor.pt')
users_cat_pro = torch.load(path2 / 'cat_properties_tensor.pt')

graph = {i: {'num_vector': [], 'cat_vector': []} for i in range(len(user_edge))}

In [91]:
for i in tqdm(range(len(user_edge))):
    if user_edge[i] != []:
        user_order = list(set(user_edge[i][0]))
        for j in range(len(user_order)):
            if user_order[j] == i:
                user_order[0], user_order[j] = user_order[j], user_order[0]
                
        uid_to_index = {x: i for i, x in enumerate(user_order)}
        tmp = [[],[]]
        for k in range(len(user_edge[i][0])):
            tmp[0].append(uid_to_index[user_edge[i][0][k]])
            tmp[1].append(uid_to_index[user_edge[i][1][k]])
        graph[i]['edge'] = tmp
        
        for l in range(len(user_order)):
            graph[i]['num_vector'].append(users_num_pro[user_order[l]])
            graph[i]['cat_vector'].append(users_cat_pro[user_order[l]])
        graph[i]['num_vector'] = torch.stack(graph[i]['num_vector'])
        graph[i]['cat_vector'] = torch.stack(graph[i]['cat_vector'])
    else:
        graph[i]['num_vector'].append(users_num_pro[i])
        graph[i]['cat_vector'].append(users_cat_pro[i])
        graph[i]['edge'] = []

np.save('data/graph.npy', graph)

100%|██████████| 229580/229580 [00:11<00:00, 20387.35it/s]


In [93]:

a = np.load('data/graph.npy', allow_pickle=True)
print(a[0])

KeyboardInterrupt: 